# CEP COVERED BY CURRENTLY ACCREDITED BODYSHOPS

In this notebook:
- We create the table `bodyshops.cep_distance` which features the distances from each CEP to each bodyshop
- We deem which CEP's are currently covered, given that they are at a distance of less than 10km from any of our currently accredited bodyshops


In [ ]:
# Imports
import pandas as pd
import numpy as np
import psycopg2, os

# Establish connection and create its cursor
try: 
    conn = psycopg2.connect(f"host={os.environ['AURORA_POSTGRES_HOST']} dbname={os.environ['AURORA_POSTGRES_DATABASE']} user={os.environ['AURORA_POSTGRES_USERNAME']} password={os.environ['AURORA_POSTGRES_PWD']}")
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)
    

---

## 1. Tables

We work with `susep.geo_info`, featuring latitude and longitude of every unique CEP code

In [5]:
cur.execute('SELECT * FROM susep.geo_info')
susep_geo_info = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])
display(susep_geo_info.head())
display(susep_geo_info.shape)
susep_geo_info.CEP.nunique()


,Id,CEP,code_tract,address_count,lat,lon,reg_susep,cep_inicial,cidade,cep_final,cod_reg,reg_decirc,cep_ini,cep_fim
0,1,76954000,110001520000004,12060,-12.047040,-62.051736,39.0,76740,FAINA,76999,48.0,DEMAIS REGI�ES DE GOI�S,76740,76999
1,777,76873868,110002305000060,11,-9.900819,-63.026831,39.0,76740,FAINA,76999,48.0,DEMAIS REGI�ES DE GOI�S,76740,76999
2,2,76955000,110007205000011,76,-12.983041,-60.952011,39.0,76740,FAINA,76999,48.0,DEMAIS REGI�ES DE GOI�S,76740,76999
3,1553,76965748,110004905000051,8,-11.436342,-61.436585,39.0,76740,FAINA,76999,48.0,DEMAIS REGI�ES DE GOI�S,76740,76999
4,3,76594000,110001505000018,9,-11.928932,-61.990097,39.0,76590,SAO MIGUEL DO ARAGUAIA,76599,48.0,DEMAIS REGI�ES DE GOI�S,76590,76599


(562192, 14)

562192

And also with `bodyshops.accredited`, with the information for each one of our accredited bodyshops, including latitude and longitude

In [6]:
cur.execute('SELECT * FROM bodyshops.accredited')
bodyshops = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])
display(bodyshops.head())
display(bodyshops.shape)


,nome_da_oficina,full_address,latitude,longitude,cep,razao_social,cnpj,telefone,responsavel,logradouro,numero,complemento,bairro,cidade,estado,inicio_recebimento,termino_recebimento
0,Scattini Funilaria,"Al,. dos Nhambiquaras 1542 Moema São Paulo SP ...",-23.6112234,-46.6636607,04090-003,Scattini Funilaria e Pintura de Autos Ltda,52.644.093/0001-90,(11) 5543-9622,Junior,"Al,. dos Nhambiquaras",1542,,Moema,São Paulo,SP,8:00:00 AM,6:00:00 PM
1,Oficina Marques (Vila Mascote),Av. Santa Catarina 2580 Vila Mascote São Paulo...,-23.6562648,-46.655485,04378-200,Oficina Marques Funilaria e Pintura Ltda,07.427.476/0001-52,(11) 5535-1903,Cae,Av. Santa Catarina,2580,,Vila Mascote,São Paulo,SP,8:00:00 AM,5:00:00 PM
2,Oficina Marques (Santo Amaro),Av. Santo Amaro 4408 Brooklin São Paulo SP 045...,-23.6216384,-46.6845705,04556-500,Marques e Marques Funilaria e Pintura Ltda,07.427.481/001-65,(11) 5535-1903,Cae,Av. Santo Amaro,4408,,Brooklin,São Paulo,SP,8:00:00 AM,5:45:00 PM
3,Ponce e Filho,R. Butiá 42 Vila Reg. Feijó São Paulo SP 03346...,-23.5588674,-46.5703805,03346-010,Ponce & Filho Ltda,54.824.826/0001-02,(11) 2965-7383,Emilio,R. Butiá,42,,Vila Reg. Feijó,São Paulo,SP,8:00:00 AM,5:30:00 PM
4,Dimas Norte,Av. Gen. Ataliba Leonel 1727 Carandiru São Pau...,-23.4999513,-46.6126825,02033-010,Fiola Reparadora de Veiciulos Eireli,17.237.668/0001-50,(11) 2904-8400,Fiola,Av. Gen. Ataliba Leonel,1727,,Carandiru,São Paulo,SP,8:00:00 AM,5:00:00 PM


(40, 17)

---

## 2. `bodyshops.cep_distance` table

This table features the distance from each bodyshops to each known CEP in meters

In [78]:
# SQL statement (check https://postgis.net/docs/ST_Distance.html for documentation on distance methods)
statement = """CREATE TABLE bodyshops.cep_distance AS (SELECT g."CEP" as cep, 
                                                              g.lon as cep_longitude, 
                                                              g.lat as cep_latitude, 
                                                              b.nome_da_oficina as bodyshop_name, 
                                                              b.longitude as bodyshop_longitude, 
                                                              b.latitude as bodyshop_latitude,
                                                              ST_Distance(ST_Transform(ST_SetSRID(ST_MakePoint(g.lon, g.lat), 4326)::geometry, 3857),
                                                                          ST_Transform(ST_SetSRID(ST_MakePoint(b.longitude, b.latitude), 4326)::geometry, 3857)) as distance_meters
                                                       FROM susep.geo_info AS g 
                                                       CROSS JOIN 
                                                       bodyshops.accredited AS b)"""
# Create the table
cur.execute(statement)

# Collec the table for analysis
cur.execute("SELECT * FROM bodyshops.cep_distance")
output = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])
output.head()


,cep,cep_longitude,cep_latitude,bodyshop_name,bodyshop_longitude,bodyshop_latitude,distance_meters
0,76954000,-62.051736,-12.04704,Scattini Funilaria,-46.6636607,-23.6112234,2.184144e+06
1,76954000,-62.051736,-12.04704,Oficina Marques (Vila Mascote),-46.655485,-23.6562648,2.188256e+06
2,76954000,-62.051736,-12.04704,Oficina Marques (Santo Amaro),-46.6845705,-23.6216384,2.183105e+06
3,76954000,-62.051736,-12.04704,Ponce e Filho,-46.5703805,-23.5588674,2.188372e+06
4,76954000,-62.051736,-12.04704,Dimas Norte,-46.6126825,-23.4999513,2.180257e+06


---
## 3. Analysis of covered bodyshops

In [79]:
output.shape

(22487680, 7)

In [80]:
output[output['distance_meters']<10000]

,cep,cep_longitude,cep_latitude,bodyshop_name,bodyshop_longitude,bodyshop_latitude,distance_meters
134880,09950000,-46.689417,-23.625896,Scattini Funilaria,-46.6636607,-23.6112234,3376.215877
134881,09950000,-46.689417,-23.625896,Oficina Marques (Vila Mascote),-46.655485,-23.6562648,5280.775858
134882,09950000,-46.689417,-23.625896,Oficina Marques (Santo Amaro),-46.6845705,-23.6216384,747.476163
134887,09950000,-46.689417,-23.625896,Quatrocentos,-46.7104603,-23.6731984,6207.402056
134893,09950000,-46.689417,-23.625896,Solução Automotiva,-46.7605456,-23.5993133,8551.297924
...,...,...,...,...,...,...,...
16443821,06780050,-46.791408,-23.624734,Celio´s Car Taboão,-46.7956835,-23.6405345,1978.045734
16443825,06780050,-46.791408,-23.624734,Jotecar,-46.7670739,-23.6393386,3238.417924
16443828,06780050,-46.791408,-23.624734,Auto Palace,-46.7170674,-23.581543,9798.759112
16443831,06780050,-46.791408,-23.624734,Volare Reparos,-46.8399564,-23.5901137,6848.099962
